# Modifying the population using complex policies

This notebook shows how complex policies can be applied to people, households, and entire populations.

In [1]:
from pam.activity import Activity, Leg
from pam.core import Household, Person, Population
from pam.policy import (
    ActivityPolicy,
    ActivityProbability,
    HouseholdPolicy,
    MoveActivityTourToHomeLocation,
    PersonAttributeFilter,
    PersonPolicy,
    PersonProbability,
    ReduceSharedActivity,
    RemoveActivity,
    apply_policies,
)
from pam.utils import minutes_to_datetime as mtdt
from pam.variables import END_OF_DAY

ModuleNotFoundError: No module named 'pam'

In [2]:
def print_simple_stats(population):
    """Print some simple population statistics."""
    time_at_home = 0
    travel_time = 0
    low_income_central_trips = 0
    high_income_central_trips = 0

    for _hid, hh in population.households.items():
        for _pid, person in hh.people.items():
            freq = person.freq

            for p in person.plan:
                if p.act == "travel":
                    duration = p.duration.seconds * freq / 3600
                    travel_time += duration

                    if p.end_location.area == "Westminster,City of London":
                        if person.attributes["inc"] == "low":
                            low_income_central_trips += freq

                        elif person.attributes["inc"] == "high":
                            high_income_central_trips += freq

                else:  # activity
                    if p.act == "home":
                        duration = p.duration.seconds * freq / 3600
                        time_at_home += duration

    print(f"Population total time at home: {time_at_home:.2f} hours")
    print(f"Population total travel time: {travel_time:.2f} hours")

## Create example Population

In [3]:
population = Population()

### Add Smith household

In [4]:
smith_id = "Smith"

Billy = Person(
    "Billy",
    freq=1,
    attributes={"age": 26, "job": "employed", "gender": "female", "key_worker": False},
)

Billy.add(Activity(1, "home", "a", start_time=mtdt(0), end_time=mtdt(8 * 60)))
Billy.add(
    Leg(1, "car", start_area="a", end_area="g", start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 20))
)
Billy.add(Activity(2, "work", "g", start_time=mtdt(8 * 60 + 20), end_time=mtdt(17 * 60)))
Billy.add(
    Leg(
        2,
        "car",
        start_area="g",
        end_area="a",
        start_time=mtdt(17 * 60),
        end_time=mtdt(17 * 60 + 25),
    )
)
Billy.add(Activity(3, "home", "a", start_time=mtdt(17 * 60 + 25), end_time=END_OF_DAY))


Bobby = Person(
    "Bobby",
    freq=1,
    attributes={"age": 6, "job": "education", "gender": "male", "key_worker": False},
)
Bobby.add(Activity(1, "home", "a", start_time=mtdt(0), end_time=mtdt(8 * 60)))
Bobby.add(Leg(1, "walk", "a", "b", start_time=mtdt(8 * 60 + 5), end_time=mtdt(8 * 60 + 30)))
Bobby.add(Activity(2, "education", "b", start_time=mtdt(8 * 60 + 30), end_time=mtdt(16 * 60)))
Bobby.add(Leg(2, "walk", "b", "c", start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 35)))
Bobby.add(Activity(3, "home", "a", start_time=mtdt(16 * 60 + 30), end_time=mtdt(18 * 60)))
Bobby.add(Leg(3, "car", "a", "b", start_time=mtdt(18 * 60), end_time=mtdt(18 * 60 + 20)))
Bobby.add(Activity(4, "shop_1", "b", start_time=mtdt(18 * 60 + 20), end_time=mtdt(18 * 60 + 50)))
Bobby.add(Leg(4, "car", "b", "b", start_time=mtdt(18 * 60 + 50), end_time=mtdt(19 * 60)))
Bobby.add(Activity(5, "shop_2", "b", start_time=mtdt(19 * 60), end_time=mtdt(19 * 60 + 50)))
Bobby.add(Leg(5, "car", "b", "a", start_time=mtdt(19 * 60 + 50), end_time=mtdt(20 * 60 + 10)))
Bobby.add(Activity(6, "home", "a", start_time=mtdt(20 * 60 + 10), end_time=END_OF_DAY))

Bradly = Person(
    "Bradly",
    freq=1,
    attributes={"age": 40, "job": "employed", "gender": "male", "key_worker": True},
)
Bradly.add(Activity(1, "home", "a", start_time=mtdt(0), end_time=mtdt(8 * 60)))

Bradly.add(Leg(1, "walk", "a", "b", start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 20)))
Bradly.add(Activity(2, "escort", "b", start_time=mtdt(8 * 60 + 20), end_time=mtdt(8 * 60 + 30)))
Bradly.add(Leg(2, "pt", "b", "b", start_time=mtdt(8 * 60 + 30), end_time=mtdt(9 * 60)))

Bradly.add(Activity(4, "work", "b", start_time=mtdt(9 * 60), end_time=mtdt(10 * 60)))
Bradly.add(Leg(3, "pt", "b", "c", start_time=mtdt(10 * 60), end_time=mtdt(10 * 60 + 20)))
Bradly.add(Activity(4, "work", "b", start_time=mtdt(10 * 60 + 20), end_time=mtdt(12 * 60)))
Bradly.add(Leg(4, "pt", "b", "c", start_time=mtdt(12 * 60), end_time=mtdt(12 * 60 + 20)))
Bradly.add(Activity(5, "work", "b", start_time=mtdt(12 * 60 + 20), end_time=mtdt(14 * 60)))
Bradly.add(Leg(5, "pt", "b", "c", start_time=mtdt(14 * 60), end_time=mtdt(14 * 60 + 20)))

Bradly.add(Activity(6, "leisure", "c", start_time=mtdt(14 * 60 + 20), end_time=mtdt(15 * 60 + 30)))
Bradly.add(Leg(4, "pt", "c", "b", start_time=mtdt(15 * 60 + 30), end_time=mtdt(16 * 60 - 10)))
Bradly.add(Activity(5, "escort", "b", start_time=mtdt(16 * 60 - 10), end_time=mtdt(16 * 60)))
Bradly.add(Leg(5, "walk", "b", "a", start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 20)))

Bradly.add(Activity(8, "home", "a", start_time=mtdt(16 * 60 + 20), end_time=mtdt(18 * 60)))
Bradly.add(Leg(8, "car", "a", "b", start_time=mtdt(18 * 60), end_time=mtdt(18 * 60 + 20)))
Bradly.add(Activity(9, "shop_1", "b", start_time=mtdt(18 * 60 + 20), end_time=mtdt(18 * 60 + 50)))
Bradly.add(Leg(9, "car", "b", "b", start_time=mtdt(18 * 60 + 50), end_time=mtdt(19 * 60)))
Bradly.add(Activity(10, "shop_2", "b", start_time=mtdt(19 * 60), end_time=mtdt(19 * 60 + 50)))
Bradly.add(Leg(10, "car", "b", "a", start_time=mtdt(19 * 60 + 50), end_time=mtdt(20 * 60 + 10)))
Bradly.add(Activity(11, "home", "a", start_time=mtdt(20 * 60 + 10), end_time=END_OF_DAY))

smiths = Household(smith_id)
for person in [Billy, Bradly, Bobby]:
    smiths.add(person)

smiths.people

In [5]:
smiths.plot()

In [6]:
population.add(smiths)

### Add Jones household

In [7]:
jones_id = "Jones"

Hugh = Person(
    "Hugh",
    freq=1,
    attributes={"age": 100, "job": "unemployed", "gender": "male", "key_worker": False},
)
Hugh.add(Activity(1, "home", "a", start_time=mtdt(0), end_time=mtdt(8 * 60)))
Hugh.add(Leg(1, "walk", "a", "b", start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 30)))
Hugh.add(Activity(2, "health", "b", start_time=mtdt(8 * 60 + 30), end_time=mtdt(10 * 60)))
Hugh.add(Leg(2, "walk", "b", "a", start_time=mtdt(10 * 60), end_time=mtdt(10 * 60 + 30)))
Hugh.add(Activity(3, "home", "a", start_time=mtdt(10 * 60 + 30), end_time=mtdt(14 * 60)))
Hugh.add(Leg(3, "walk", "a", "b", start_time=mtdt(14 * 60), end_time=mtdt(14 * 60 + 30)))
Hugh.add(Activity(4, "health", "b", start_time=mtdt(14 * 60 + 30), end_time=mtdt(16 * 60)))
Hugh.add(Leg(4, "walk", "b", "a", start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 30)))
Hugh.add(Activity(5, "home", "a", start_time=mtdt(16 * 60 + 30), end_time=END_OF_DAY))

Bridget = Person(
    "Bridget",
    freq=1,
    attributes={"age": 35, "job": "employed", "gender": "female", "key_worker": False},
)
Bridget.add(Activity(1, "home", "a", start_time=mtdt(0), end_time=mtdt(8 * 60)))
Bridget.add(Leg(1, "walk", "a", "b", start_time=mtdt(8 * 60), end_time=mtdt(8 * 60 + 5)))
Bridget.add(Activity(2, "escort", "b", start_time=mtdt(8 * 60 + 5), end_time=mtdt(10 * 60 + 30)))
Bridget.add(Leg(2, "pt", "b", "c", start_time=mtdt(10 * 60 + 30), end_time=mtdt(11 * 60)))
Bridget.add(Activity(3, "work", "c", start_time=mtdt(11 * 60), end_time=mtdt(16 * 60)))
Bridget.add(Leg(3, "pt", "c", "a", start_time=mtdt(16 * 60), end_time=mtdt(16 * 60 + 20)))
Bridget.add(Activity(4, "home", "a", start_time=mtdt(16 * 60 + 20), end_time=mtdt(17 * 60 + 20)))
Bridget.add(Leg(4, "pt", "c", "a", start_time=mtdt(17 * 60 + 20), end_time=mtdt(17 * 60 + 50)))
Bridget.add(Activity(5, "shop", "a", start_time=mtdt(17 * 60 + 50), end_time=mtdt(18 * 60 + 30)))
Bridget.add(Leg(5, "pt", "c", "a", start_time=mtdt(18 * 60 + 30), end_time=mtdt(18 * 60 + 50)))
Bridget.add(Activity(6, "home", "a", start_time=mtdt(18 * 60 + 50), end_time=END_OF_DAY))

jones = Household(jones_id)
for person in [Hugh, Bridget]:
    jones.add(person)

jones.people

In [8]:
jones.plot()

In [9]:
population.add(jones)

In [10]:
print_simple_stats(population)

## PAM Complex Policies

Based on [link](https://docs.google.com/spreadsheets/d/1FQMa7dLe2cv1NEZnbu5cZo3v07tKXINwvOaLQYoEp-M/edit#gid=0)

In [11]:
def discrete_joint_distribution_sampler(obj, mapping, distribution):
    """
    Randomly sample from a joint distribution based some discrete features.
    Where features are a dictionary structure of features, eg: {'gender':'female'}
    Distribution is a nested dict of probabilities based on possible features, eg:
    {'0-0': {'male': 0, 'female': 0},... , '90-120': {'male': 1, 'female': 1}}
    Mapping provides the feature name for each level of the distribution, eg:
    ['age', 'gender'].
    """
    p = distribution
    for key in mapping:
        value = obj.attributes.get(key)
        if value is None:
            msg = f"Cannot find mapping: {key} in sampling features: {obj.attributes}"
            raise KeyError(msg)
        p = p.get(value)
        if p is None:
            msg = f"Cannot find feature for {key}: {value} in distribution: {p}"
            raise KeyError(msg)

    return p

In [12]:
vulnerable_mapping = ["age", "gender"]
vulnerable_distribution = dict(
    zip(
        list(range(101)),
        [{"male": i / 100, "female": i / 100, "other": i / 100} for i in range(101)],
    )
)

In [13]:
dict(list(vulnerable_distribution.items())[0:15])

In [14]:
discrete_joint_distribution_sampler(Bobby, vulnerable_mapping, vulnerable_distribution)

### Household Quarantine

Probabilistically apply quarantine to a household (remove all activities - stay at home) 

#### Person-based, all people equal

If you have a probability of any person having to be quarantined

In [15]:
policy_quarantine = HouseholdPolicy(
    RemoveActivity(["work", "health", "leisure", "escort", "shop", "education"]),
    PersonProbability(0.0000000000001),
)

In [16]:
q_pop = apply_policies(population, [policy_quarantine])
print_simple_stats(q_pop)

In [17]:
q_pop.households["Smith"].plot()

In [18]:
q_pop.households["Jones"].plot()

#### Using joint distribution

In [19]:
policy_quarantine = HouseholdPolicy(
    RemoveActivity(["work", "health", "escort", "leisure", "shop", "education"]),
    PersonProbability(
        discrete_joint_distribution_sampler,
        {"mapping": vulnerable_mapping, "distribution": vulnerable_distribution},
    ),
)

In [20]:
q_pop = apply_policies(population, [policy_quarantine])
print_simple_stats(q_pop)

In [21]:
q_pop.households["Jones"].plot()

#### Chaining probabilities

In [22]:
policy_quarantine = HouseholdPolicy(
    RemoveActivity(["work", "health", "escort", "leisure", "shop", "education"]),
    [
        PersonProbability(0.5),
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": vulnerable_mapping, "distribution": vulnerable_distribution},
        ),
    ],
)

In [23]:
q_pop = apply_policies(population, [policy_quarantine])
print_simple_stats(q_pop)

In [24]:
q_pop.households["Jones"].plot()

### Remove Education

Probabilistically remove education activities from a person and escort from people in the same household

In [25]:
edu_mapping = ["job"]
edu_distribution = {"employed": 0, "unemployed": 0, "education": 1}

In [26]:
key_mapping = ["key_worker"]
key_distribution = {True: 0, False: 1}

In [27]:
edu_distribution, key_distribution

In [28]:
policy_remove_education_and_escort = HouseholdPolicy(
    RemoveActivity(["education", "escort"]),
    [
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": edu_mapping, "distribution": edu_distribution},
        ),
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": key_mapping, "distribution": key_distribution},
        ),
    ],
)

In [29]:
edu_pop = apply_policies(population, [policy_remove_education_and_escort])
print_simple_stats(edu_pop)

In [30]:
edu_pop.households["Smith"].people["Bradly"].plan.print()

In [31]:
q_pop.households["Smith"].plot()

In [32]:
edu_pop.households["Smith"].plot()

In [33]:
edu_pop.households["Jones"].plot()

In [34]:
edu_pop.households["Jones"].people["Bridget"].plan.print()

#### PersonAttributeFilter
You can also use the `modify.PersonAttributeFilter` to only affect people with certain attributes.

In [35]:
def condition_job_education(val):
    return val == "education"


policy_remove_education_and_escort = HouseholdPolicy(
    RemoveActivity(["education", "escort"]),
    PersonProbability(
        discrete_joint_distribution_sampler,
        {"mapping": key_mapping, "distribution": key_distribution},
    ),
    PersonAttributeFilter({"job": condition_job_education}),
)

In [36]:
apply_policies(population, [policy_remove_education_and_escort]).households["Smith"].plot()

In [37]:
policy_remove_education_and_escort = HouseholdPolicy(
    RemoveActivity(["education", "escort"]),
    [
        ActivityProbability(["education"], 0.9999),
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": edu_mapping, "distribution": edu_distribution},
        ),
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": key_mapping, "distribution": key_distribution},
        ),
    ],
)

In [38]:
apply_policies(population, [policy_remove_education_and_escort]).households["Smith"].plot()

### Remove Leisure Activities

Remove all leisure activities

In [39]:
policy_remove_leisure = PersonPolicy(RemoveActivity(["leisure"]), PersonProbability(1.0))

In [40]:
lei_pop = apply_policies(population, [policy_remove_leisure])
print_simple_stats(lei_pop)

In [41]:
lei_pop.households["Smith"].plot()

### Unemployment/Furlough

Probabilistically remove all work activities from a person

In [42]:
key_mapping = ["key_worker"]
key_distribution = {True: 0, False: 1}

In [43]:
policy_unemployment_and_furlough = PersonPolicy(
    RemoveActivity(["work"]),
    [
        PersonProbability(
            discrete_joint_distribution_sampler,
            {"mapping": key_mapping, "distribution": key_distribution},
        )
    ],
)

In [44]:
fur_pop = apply_policies(population, [policy_unemployment_and_furlough])
print_simple_stats(fur_pop)

In [45]:
fur_pop.households["Smith"].plot()

In [46]:
fur_pop.households["Jones"].plot()

In [47]:
policy_reduced_work = ActivityPolicy(RemoveActivity(["work"]), ActivityProbability(["work"], 0.5))

apply_policies(population, [policy_reduced_work]).households["Smith"].plot()

## Remove Shopping 

Probabilistically remove **individual** shopping activities from a person

In [48]:
policy_remove_shopping = PersonPolicy(RemoveActivity(["shop"]), ActivityProbability(["shop"], 1.0))

In [49]:
shop_pop = apply_policies(population, [policy_remove_shopping])
print_simple_stats(shop_pop)

In [50]:
shop_pop.households["Jones"].people["Bridget"].plot()

### Reducing shared activities

In [51]:
policy_reduce_shopping_activities = HouseholdPolicy(
    ReduceSharedActivity(["shop_1", "shop_2", "shop"]),
    ActivityProbability(["shop_1", "shop_2", "shop"], 1.0),
)

In [52]:
population["Smith"].shared_activities()

In [53]:
shop_reduce_pop = apply_policies(population, [policy_reduce_shopping_activities])
print_simple_stats(shop_reduce_pop)

In [54]:
shop_reduce_pop.households["Smith"].plot()

In [55]:
shop_reduce_pop.households["Smith"].people["Bobby"].print()

### Moving Shopping tours


In [56]:
policy_move_shopping_tours = PersonPolicy(
    MoveActivityTourToHomeLocation(["shop_1", "shop_2"]),
    [ActivityProbability(["shop_1", "shop_2"], 1.0)],
)

In [57]:
shop_tour_pop = apply_policies(population, [policy_move_shopping_tours])
print_simple_stats(shop_tour_pop)

In [58]:
shop_tour_pop.households["Smith"].plot()

In [59]:
# above is equivalent to
policy_move_shopping_tours = PersonPolicy(
    MoveActivityTourToHomeLocation(["shop_1", "shop_2"]),
    [ActivityProbability(["shop_1"], 1.0), ActivityProbability(["shop_2"], 1.0)],
)

In [60]:
shop_tour_pop = apply_policies(population, [policy_move_shopping_tours])
print_simple_stats(shop_tour_pop)

In [61]:
shop_tour_pop.households["Smith"].plot()

In [62]:
shop_tour_pop.households["Smith"].people["Bradly"].print()

## All together now!

In [63]:
all_together_pop = apply_policies(
    population,
    [
        policy_quarantine,
        policy_remove_education_and_escort,
        policy_remove_leisure,
        policy_unemployment_and_furlough,
        policy_remove_shopping,
    ],
)

print_simple_stats(all_together_pop)